---
title: Contributor Metrics -- pyOpenSci
subtitle: Counts of issue and pull request activity on GitHub
license:
  code: BSD-3-Clause
---

In [3]:
import warnings
from pathlib import Path

import altair as alt
import pandas as pd

# This is a local module that stores the plot theme
from pyosmetrics.plot_theme import load_poppins_font, register_and_enable_poppins_theme

pd.options.mode.chained_assignment = None
pd.options.future.infer_string = True

# Suppress all warnings
warnings.filterwarnings("ignore")

# Load the & register Poppins theme
load_poppins_font()
register_and_enable_poppins_theme()

In [4]:
# Get the current notebook directory
current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
data_dir = parent_dir / "_data"
all_files = list(data_dir.glob("*"))
dfs = [pd.read_csv(file, parse_dates=["item_opened_by"]) for file in all_files]
df = pd.concat(dfs, ignore_index=True)

## All contributions to date 

The plot below shows total contributions to pyOpenSci. To create this plot we:

1. Removed any contributions from bots
2. Stratified the data to show both core staff contributions vs. non-staff contributions. 


### Staff vs. volunteer contributions

Non-staff contributions tend to grow between May and August which cover Q3 and Q4 when we travel to meetings such as SciPy and PyCon and host sprint and other events. Q4 is typically slower given the holiday season.  

In [5]:
df["quarter"] = df["item_opened_by"].dt.to_period("Q")

# Group by quarter and contrib_type (staff vs non-staff)
df_quarters = df.groupby(["quarter", "contrib_type"]).size().reset_index(name="count")
df_quarters["quarter"] = df_quarters["quarter"].astype(str)

# Create a new column for filtered labels
df_quarters["quarter_label"] = df_quarters["quarter"]
df_quarters["year"] = df_quarters["quarter"].str[:4]
df_quarters["qtr"] = df_quarters["quarter"].str[-2:]
df_quarters.reset_index(inplace=True)

In [6]:
# Create the Altair chart
chart = (
    alt.Chart(df_quarters)
    .mark_bar()
    .encode(
        x=alt.X(
            "quarter:O",
            title="Quarter",
            axis=alt.Axis(labelExpr='datum.index % 3 == 0 ? datum.label : ""')),
        y=alt.Y("count:Q", stack=True, title="Count"),
        color=alt.Color(
            "contrib_type:N",
            title="Contributor Type (Staff vs Non-Staff)",
            scale=alt.Scale(range=["#81c0aa", "#735fab"]),
        ),
        tooltip=["quarter", "contrib_type", "count"],
    )
    .properties(title="Staff vs. Non Staff Contributor by Quarter")
    .configure_legend(
        orient="top",
        title=None,
        legendX=-550,
        legendY=-30,
    )
    .interactive()
)

chart

alt.Chart(...)